In [1]:
import pandas as pd
print('pd')

pd


In [2]:
#  remove side bars

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

In [2]:
import sys
from IPython.core.interactiveshell import InteractiveShell
import pickle
from itertools import chain, permutations
from tqdm import tqdm_notebook
import time
import datetime
import itertools
import random
import seaborn as sns
from matplotlib.ticker import FormatStrFormatter
import matplotlib.pyplot as plt
import scipy
from numpy import inf
import numpy as np
import pandas as pd
import warnings

%pwd  #  where the notebook is located
!jupyter nbextension enable - -py widgetsnbextension

warnings.filterwarnings("ignore")


%matplotlib inline

InteractiveShell.ast_node_interactivity = 'all'

sys.executable  # env
sys.version

'C:\\Users\\antoh\\Downloads'

Unrecognized alias: 'py', it will have no effect.
Enabling notebook extension -...
      - Validating: problems found:
        - require?  X -


'C:\\Users\\antoh\\anaconda3\\python.exe'

'3.7.10 (default, Feb 26 2021, 13:06:18) [MSC v.1916 64 bit (AMD64)]'

# Желтое такси в Нью-Йорке

Нью-Йоркская комиссия по такси и лимузинам (TLC) предоставляет подробные анонимизированные данные о поездках клиентов с 2009 года. Машины, выкрашенные жёлтым, имеют право брать пассажиров на улицах в любом из пяти боро города.

<img src="taxi.jpg" width="600">

Данные разбиты на файлы по месяцам. В каждом из файлов содержится следующая информация о поездках:

* время начала поездки
* время окончания поездки
* долгота и широта точки начала поездки
* долгота и широта точки окончания поездки
* количество пассажиров
* расстояние по счётчику
* тип тарифа (одна из шести категорий)
* способ оплаты (одна из шести категорий)
* стоимость поездки по счётчику
* налог на счётчик
* доплата за поездки в пиковые часы и ночью
* доплата за проезд по платным дорогам
* доплата за проезд, взимаемая с каждой поездки с января 2015
* размер чаевых
* общая стоимость поездки
* провайдер данных (одна из двух категорий)
* бинарный флаг, показывающий, были ли данные о поездке получены немедленно после её окончания, или какое-то время хранились в памяти автомобиля.

__Задача проекта__ — научиться предсказывать количество поездок в ближайшие часы в каждом районе Нью-Йорка; для простоты мы определим прямоугольные районы. Для того, чтобы её решить, сырые данные _необходимо агрегировать по часам и районам_. Агрегированные данные будут представлять собой почасовые временные ряды с количествами поездок из каждого района.

Задачу прогнозирования таких рядов можно решать с помощью авторегрессионных моделей, прогнозируя каждый ряд независимо. Ряды явно имеют сложную сезонность — суточную, недельную и годовую, поэтому для их моделирования понадобится использовать модель ARIMA с дополнительной регрессией на внешние признаки.

Чтобы улучшить такую модель, можно попытаться учесть:

* взаимосвязи между рядами
* дополнительные параметры поездок, которые можно посчитать по исходным данным
* внешние календарные и географические признаки

# Week 1

Для сдачи задания этой недели вам нужно агрегировать только данные за май 2016, но, когда задание будет сдано, не забудьте запустить ваш агрегирующий скрипт на всех остальных месяцах, которые вы собираетесь использовать при прогнозировании.

Чтобы сдать задание, выполните следующую последовательность действий.

1. Загрузите данные за май 2016 года.

2. Проведите фильтрацию: удалите поездки с
 * нулевой длительностью
 * нулевым количеством пассажиров
 * нулевым расстоянием поездки по счётчику
 * координатами начала, не попадающими в прямоугольник Нью-Йорка

Можно придумать ещё какие-то критерии для фильтрации данных; тем не менее, не стоит применять дополнительные фильтры: начиная с четвёртой недели вам предстоит сравнивать качество ваших прогнозов с качеством прогнозов других слушателей, и, чтобы это сравнение было корректным, нужно, чтобы данные у всех были предобработаны одинаково. 

3. Агрегируйте данные за май 2016 года по часам и областям (можно использовать функцию stats.binned_statistic_2d, с которой вы сталкивались в заданиях четвёртого курса). Не забудьте проверить, что идентификаторы, которые вы присваиваете ячейкам, соответствуют приложенному выше файлу.

4. Выведите первые строки полученной таблицы с указанием времени, идентификатора зоны и количества поездок.

5. Постройте график количества поездок жёлтого такси из ячейки, содержащей Empire State Building.

6. Посчитайте, сколько в мае 2016 было пар час-ячейка, для которых не было совершено ни одной поездки.

7. Загрузите ноутбук в форму ниже

In [2]:
#  display the result side by side if print row

from IPython.display import display, HTML

CSS = """
.output {
    flex-direction: column;
}
"""

HTML('<style>{}</style>'.format(CSS))

In [4]:
ny_loc = [[-74.25559, -73.70001], [40.49612, 40.91553]]

In [3]:
df = pd.read_csv('yellow_tripdata_2016-05.csv',
                 header=0,
                 skipinitialspace=True,
                 parse_dates=[1,2],
                 date_parser=lambda x: datetime.datetime.strptime(x, '%Y-%m-%d ')
             

                )
df.info()
df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11836853 entries, 0 to 11836852
Data columns (total 19 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   VendorID               int64  
 1   tpep_pickup_datetime   object 
 2   tpep_dropoff_datetime  object 
 3   passenger_count        int64  
 4   trip_distance          float64
 5   pickup_longitude       float64
 6   pickup_latitude        float64
 7   RatecodeID             int64  
 8   store_and_fwd_flag     object 
 9   dropoff_longitude      float64
 10  dropoff_latitude       float64
 11  payment_type           int64  
 12  fare_amount            float64
 13  extra                  float64
 14  mta_tax                float64
 15  tip_amount             float64
 16  tolls_amount           float64
 17  improvement_surcharge  float64
 18  total_amount           float64
dtypes: float64(12), int64(4), object(3)
memory usage: 1.7+ GB


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,1,2016-05-01 00:00:00,2016-05-01 00:17:31,1,3.60,-73.985901,40.768040,1,N,-73.983986,40.730099,1,15.0,0.5,0.5,1.50,0.0,0.3,17.80
1,2,2016-05-01 00:00:00,2016-05-01 00:07:31,1,1.68,-73.991577,40.744751,1,N,-73.975700,40.765469,1,7.5,0.5,0.5,0.88,0.0,0.3,9.68
2,2,2016-05-01 00:00:00,2016-05-01 00:07:01,6,1.09,-73.993073,40.741573,1,N,-73.980995,40.744633,1,6.5,0.5,0.5,1.56,0.0,0.3,9.36
3,2,2016-05-01 00:00:00,2016-05-01 00:19:47,1,4.21,-73.991943,40.684601,1,N,-74.002258,40.733002,1,17.0,0.5,0.5,3.66,0.0,0.3,21.96
4,2,2016-05-01 00:00:00,2016-05-01 00:06:39,1,0.56,-74.005280,40.740192,1,N,-73.997498,40.737564,1,6.0,0.5,0.5,1.46,0.0,0.3,8.76
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11836848,2,2016-05-27 12:24:58,2016-05-27 12:39:38,1,0.00,0.000000,0.000000,1,N,0.000000,0.000000,2,9.5,0.5,0.5,0.00,0.0,0.3,10.80
11836849,2,2016-05-27 12:44:54,2016-05-27 12:48:17,1,0.00,0.000000,0.000000,1,N,0.000000,0.000000,2,4.0,0.5,0.5,0.00,0.0,0.3,5.30
11836850,2,2016-05-27 12:48:51,2016-05-27 13:40:56,1,0.00,0.000000,0.000000,1,N,0.000000,0.000000,2,28.5,0.5,0.5,0.00,0.0,0.3,29.80
11836851,2,2016-05-27 14:18:21,2016-05-27 14:38:13,1,0.00,0.000000,0.000000,1,N,0.000000,0.000000,2,12.0,0.5,0.5,0.00,0.0,0.3,13.30


In [8]:
df[['tpep_dropoff_datetime']]

,tpep_dropoff_datetime
0,2016-05-01 00:17:31
1,2016-05-01 00:07:31
2,2016-05-01 00:07:01
3,2016-05-01 00:19:47
4,2016-05-01 00:06:39
...,...
11836848,2016-05-27 12:39:38
11836849,2016-05-27 12:48:17
11836850,2016-05-27 13:40:56
11836851,2016-05-27 14:38:13
